In [ ]:
import re
import math
import torch
import numpy as np
from random import *
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

In [ ]:
text = (
    'Hello, how are you? I am Romeo.\n' # R
    'Hello, Romeo My name is Juliet. Nice to meet you.\n' # J
    'Nice meet you too. How are you today?\n' # R
    'Great. My baseball team won the competition.\n' # J
    'Oh Congratulations, Juliet\n' # R
    'Thank you Romeo\n' # J
    'Where are you going today?\n' # R
    'I am going shopping. What about you?\n' # J
    'I am going to visit my grandmother. she is not very well' # R
)

In [ ]:
sentences = re.sub("[.,!?\\-]", '', text.lower()).split('\n')

In [ ]:
word_list = list(set(" ".join(sentences).split()))

In [ ]:
word2idx = {'[PAD]' : 0, '[CLS]' : 1, '[SEP]' : 2, '[MASK]' : 3}

In [ ]:
for i, w in enumerate(word_list):
    word2idx[w] = i + 4
idx2word = {i: w for i, w in enumerate(word2idx)}
vocab_size = len(word2idx)


In [ ]:
token_list = list()
for sentence in sentences:
  arr = [word2idx[s]for s in sentence.split()]
  token_list.append(arr)

In [ ]:
token_list

[[19, 12, 30, 39, 4, 21, 32],
 [19, 32, 13, 11, 6, 7, 29, 17, 36, 39],
 [29, 36, 39, 9, 12, 30, 39, 31],
 [25, 13, 27, 16, 24, 37, 23],
 [18, 14, 7],
 [15, 39, 32],
 [34, 30, 39, 35, 31],
 [4, 21, 35, 5, 10, 38, 39],
 [4, 21, 35, 17, 22, 13, 26, 33, 6, 28, 20, 8]]

In [ ]:
# BERT Parameters
maxlen = 30
batch_size = 6
max_pred = 5 # max tokens of prediction
n_layers = 6
n_heads = 12
d_model = 768
d_ff = 768*4 # 4*d_model, FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_segments = 2

In [ ]:
def make_data():
  batch = []
  positive = negative =0
  while positive != batch_size/2 or negative != batch_size/2:
    tokens_a_index, tokens_b_index = randrange(len(sentences)), randrange(len(sentences))
    tokens_a, tokens_b = token_list[tokens_a_index], token_list[tokens_b_index]
    input_ids = [word2idx['[CLS]']] + tokens_a + [word2idx['[SEP]']] + tokens_b +[word2idx['[SEP]']]
    segment_ids = [0] * (1 + len(tokens_a)+1)+[1]*(len(tokens_b)+1)

    #MLM
    n_pred = min(max_pred, max(1, int(len(input_ids)*0.15)))
    can_maked_pos = [i for i , token in enumerate(input_ids)
                    if token != word2idx['[CLS]'] and token != word2idx['[SEP]']]
    shuffle(can_maked_pos)
    masked_tokens, masked_pos = [] , []
    for pos in can_maked_pos[:n_pred]:
      masked_pos.append(pos)
      masked_tokens.append(input_ids[pos])
      if random() < 0.8:
        input_ids[pos] = word2idx['[MASK]']
      elif random() >0.9:
        index = randint(0,vocab_size-1)
        while index <4:
          index = randint(0, vocab_size-1)
        input_ids[pos] = index
    

    #Zero Paddings
    n_pad = maxlen - len(input_ids)
    input_ids.extend([0]*n_pad)
    segment_ids.extend([0]*n_pad)

    #Zero Padding(100%-15%)tokens
    if max_pred > n_pred:
      n_pad = max_pred - n_pred
      masked_tokens.extend([0]*n_pad)
      masked_pos.extend([0]*n_pad)

    if tokens_a_index +1 == tokens_b_index and positive < batch_size/2:
      batch.append([input_ids, segment_ids , masked_tokens, masked_pos, True])
      positive +=1
    elif tokens_a_index +1 != tokens_b_index and negative < batch_size/2:
      batch.append([input_ids, segment_ids , masked_tokens, masked_pos, False])
      negative +=1
  return batch
  # Proprecessing Finished

In [ ]:
batch = make_data()
input_ids, segment_ids, masked_tokens, masked_pos, isNext = zip(*batch)
input_ids, segment_ids, masked_tokens, masked_pos, isNext = \
    torch.LongTensor(input_ids),  torch.LongTensor(segment_ids), torch.LongTensor(masked_tokens),\
    torch.LongTensor(masked_pos), torch.LongTensor(isNext)

class MyDataSet(Data.Dataset):
  def __init__(self, input_ids, segment_ids, masked_tokens, masked_pos, isNext):
    self.input_ids = input_ids
    self.segment_ids = segment_ids
    self.masked_tokens = masked_tokens
    self.masked_pos = masked_pos
    self.isNext = isNext
  
  def __len__(self):
    return len(self.input_ids)
  
  def __getitem__(self, idx):
    return self.input_ids[idx], self.segment_ids[idx], self.masked_tokens[idx], self.masked_pos[idx], self.isNext[idx]

loader = Data.DataLoader(MyDataSet(input_ids, segment_ids, masked_tokens, masked_pos, isNext), batch_size, True)

In [ ]:
def get_attn_pad_mask(seq_q, seq_k):
    batch_size, seq_len = seq_q.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_q.data.eq(0).unsqueeze(1)  # [batch_size, 1, seq_len]
    return pad_attn_mask.expand(batch_size, seq_len, seq_len)  # [batch_size, seq_len, seq_len]

def gelu(x):
    """
      Implementation of the gelu activation function.
      For information: OpenAI GPT's gelu is slightly different (and gives slightly different results):
      0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
      Also see https://arxiv.org/abs/1606.08415
    """
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

class Embedding(nn.Module):
    def __init__(self):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
        self.pos_embed = nn.Embedding(maxlen, d_model)  # position embedding
        self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, seg):
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long)
        pos = pos.unsqueeze(0).expand_as(x)  # [seq_len] -> [batch_size, seq_len]
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        return self.norm(embedding)

class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size, n_heads, seq_len, seq_len]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context

class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, d_v * n_heads)
    def forward(self, Q, K, V, attn_mask):
        # q: [batch_size, seq_len, d_model], k: [batch_size, seq_len, d_model], v: [batch_size, seq_len, d_model]
        residual, batch_size = Q, Q.size(0)
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # q_s: [batch_size, n_heads, seq_len, d_k]
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # k_s: [batch_size, n_heads, seq_len, d_k]
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # v_s: [batch_size, n_heads, seq_len, d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size, n_heads, seq_len, seq_len]

        # context: [batch_size, n_heads, seq_len, d_v], attn: [batch_size, n_heads, seq_len, seq_len]
        context = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v) # context: [batch_size, seq_len, n_heads, d_v]
        output = nn.Linear(n_heads * d_v, d_model)(context)
        return nn.LayerNorm(d_model)(output + residual) # output: [batch_size, seq_len, d_model]

class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        # (batch_size, seq_len, d_model) -> (batch_size, seq_len, d_ff) -> (batch_size, seq_len, d_model)
        return self.fc2(gelu(self.fc1(x)))

class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size, seq_len, d_model]
        return enc_outputs

class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.embedding = Embedding()
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.Dropout(0.5),
            nn.Tanh(),
        )
        self.classifier = nn.Linear(d_model, 2)
        self.linear = nn.Linear(d_model, d_model)
        self.activ2 = gelu
        # fc2 is shared with embedding layer
        embed_weight = self.embedding.tok_embed.weight
        self.fc2 = nn.Linear(d_model, vocab_size, bias=False)
        self.fc2.weight = embed_weight

    def forward(self, input_ids, segment_ids, masked_pos):
        output = self.embedding(input_ids, segment_ids) # [bach_size, seq_len, d_model]
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids) # [batch_size, maxlen, maxlen]
        for layer in self.layers:
            # output: [batch_size, max_len, d_model]
            output = layer(output, enc_self_attn_mask)
        # it will be decided by first token(CLS)
        h_pooled = self.fc(output[:, 0]) # [batch_size, d_model]
        logits_clsf = self.classifier(h_pooled) # [batch_size, 2] predict isNext

        masked_pos = masked_pos[:, :, None].expand(-1, -1, d_model) # [batch_size, max_pred, d_model]
        print(output)
        print(masked_pos)
        h_masked = torch.gather(output, 1, masked_pos) # masking position [batch_size, max_pred, d_model]
        print(h_masked)
        h_masked = self.activ2(self.linear(h_masked)) # [batch_size, max_pred, d_model]
        logits_lm = self.fc2(h_masked) # [batch_size, max_pred, vocab_size]
        return logits_lm, logits_clsf
model = BERT()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adadelta(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(50):
    for input_ids, segment_ids, masked_tokens, masked_pos, isNext in loader:
      logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
      loss_lm = criterion(logits_lm.view(-1, vocab_size), masked_tokens.view(-1)) # for masked LM
      loss_lm = (loss_lm.float()).mean()
      loss_clsf = criterion(logits_clsf, isNext) # for sentence classification
      loss = loss_lm + loss_clsf
      if (epoch + 1) % 10 == 0:
          print('Epoch:', '%04d' % (epoch + 1), 'loss =', '{:.6f}'.format(loss))
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

流式输出内容被截断，只能显示最后 5000 行内容。
         [-0.3219,  0.0321,  0.1849,  ..., -0.0631,  0.4050,  0.1830],
         [-0.3219,  0.0321,  0.1849,  ..., -0.0631,  0.4050,  0.1830],
         [-0.3219,  0.0321,  0.1849,  ..., -0.0631,  0.4050,  0.1830]],

        [[-0.0616,  0.1654, -0.2143,  ..., -0.0891,  0.2831,  0.0709],
         [ 0.0495, -0.1213,  0.3313,  ...,  0.1993,  0.3976,  0.0480],
         [-0.2760,  0.0550,  0.1923,  ..., -0.0529,  0.4345,  0.1412],
         [-0.2760,  0.0550,  0.1923,  ..., -0.0529,  0.4345,  0.1412],
         [-0.2760,  0.0550,  0.1923,  ..., -0.0529,  0.4345,  0.1412]],

        [[-0.1649,  0.0099,  0.2273,  ..., -0.1322,  0.3839,  0.0846],
         [-0.3046, -0.0036,  0.1812,  ..., -0.1089,  0.4033,  0.2131],
         [-0.3046, -0.0036,  0.1812,  ..., -0.1089,  0.4033,  0.2131],
         [-0.3046, -0.0036,  0.1812,  ..., -0.1089,  0.4033,  0.2131],
         [-0.3046, -0.0036,  0.1812,  ..., -0.1089,  0.4033,  0.2131]],

        [[-0.0312,  0.1878,  0.0114,  ..., -

In [ ]:
# Predict mask tokens ans isNext
input_ids, segment_ids, masked_tokens, masked_pos, isNext = batch[1]
print(text)
print('================================')
print([idx2word[w] for w in input_ids if idx2word[w] != '[PAD]'])

logits_lm, logits_clsf = model(torch.LongTensor([input_ids]), \
                 torch.LongTensor([segment_ids]), torch.LongTensor([masked_pos]))
logits_lm = logits_lm.data.max(2)[1][0].data.numpy()
print('masked tokens list : ',[pos for pos in masked_tokens if pos != 0])
print('predict masked tokens list : ',[pos for pos in logits_lm if pos != 0])

logits_clsf = logits_clsf.data.max(1)[1].data.numpy()[0]
print('isNext : ', True if isNext else False)
print('predict isNext : ',True if logits_clsf else False)

Hello, how are you? I am Romeo.
Hello, Romeo My name is Juliet. Nice to meet you.
Nice meet you too. How are you today?
Great. My baseball team won the competition.
Oh Congratulations, Juliet
Thank you Romeo
Where are you going today?
I am going shopping. What about you?
I am going to visit my grandmother. she is not very well
['[CLS]', 'nice', 'meet', 'you', 'too', 'how', 'are', 'you', 'today', '[SEP]', 'i', 'am', 'going', 'to', 'visit', 'my', 'grandmother', 'she', 'is', 'not', '[MASK]', 'well', '[SEP]']
tensor([[[-0.5340,  0.0063,  0.2120,  ...,  0.1232,  0.2325,  0.3175],
         [-0.1474, -0.0600,  0.0355,  ...,  0.1686,  0.2335,  0.2221],
         [-0.3085, -0.0393,  0.1274,  ...,  0.1118,  0.3468,  0.2360],
         ...,
         [-0.2801, -0.0868, -0.1114,  ...,  0.0705,  0.1392, -0.0316],
         [ 0.0372, -0.1657, -0.1266,  ...,  0.0145,  0.0438,  0.2006],
         [-0.3260, -0.2192,  0.0565,  ...,  0.1123,  0.2435,  0.0683]]],
       grad_fn=<AddBackward0>)
tensor([[[11, 11